# Demo 5

## Load dataset

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
import tensorflow as tf

X, y = make_moons(n_samples=200, noise=0.1)
y_hot = tf.keras.utils.to_categorical(y, num_classes=2)  # one-hot encoded labels

c = ['#1f77b4' if y_ == 0 else '#ff7f0e' for y_ in y]  # colours for each class
plt.axis('off')
plt.scatter(X[:, 0], X[:, 1], c=c)
plt.show()

## Set up QNode

In [ ]:
import pennylane as qml

n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]

## Convert to Keras layer

In [ ]:
n_layers = 3
weight_shapes = {"weights": (n_layers, n_qubits, 3)}

In [ ]:
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits)

## Create model

In [ ]:
clayer_1 = tf.keras.layers.Dense(2)
clayer_2 = tf.keras.layers.Dense(2, activation="softmax")
model = tf.keras.models.Sequential([clayer_1, qlayer, clayer_2])

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.2)
model.compile(opt, loss='mae', metrics=['accuracy'])

## Train

In [ ]:
X = X.astype("float32")
y_hot = y_hot.astype("float32")
model.fit(X, y_hot, epochs=6, batch_size=5, validation_split=0.25)

## Validate

In [ ]:
X_val, y_val = make_moons(n_samples=100, noise=0.1)

In [ ]:
y_hot_pred = model.predict(X_val)
y_pred = tf.argmax(y_hot_pred, axis=1)

In [ ]:
acc = tf.keras.metrics.Accuracy()

In [ ]:
acc(y_pred, y_val)

In [ ]:
c = ['#1f77b4' if y_ == 0 else '#ff7f0e' for y_ in y_pred]  # colours for each class
plt.axis('off')
plt.scatter(X_val[:, 0], X_val[:, 1], c=c)
plt.show()